In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
def mean(df_column):
        return(np.sum(df_column)/len(df_column))
                
def median(df_column):
        df_column = df_column.sort_values(ascending=True).reset_index(drop=True)
        if len(df_column) % 2 == 0:
                return ((df_column[len(df_column)//2]+df_column[len(df_column)//2 - 1])/2)
        else: return df_column[len(df_column)//2]

def mode(df_column):
        freq = {}
        for elem in df_column:
                if elem in freq.keys():
                        freq[elem] += 1
                else : freq[elem] = 1
        return max(freq, key=freq.get)

def tendanceCentrale(df_column): #retourne la moyenne, mode et mediane ainsi que la symétrie
        tendances = []
        mean_ = mean(df_column)
        mode_ = mode(df_column)
        median_ = median(df_column)
        tendances.append(mean_)
        tendances.append(mode_)
        tendances.append(median_)
        if round(mean_,1) == round(mode_,1) and round(mean_,1) == round(median_,1) and round(median_,1) == round(mode_,1):
                tendances.append("Symétrique")
        elif round(mean_,1) > round(median_,1) and round(median_,1) > round(mode_,1):
                tendances.append("Positivement")
        elif round(mean_,1) < round(median_,1) and round(median_,1) < round(mode_,1):
                tendances.append("Négativement")
        else: tendances.append("Inindentifiée")
        return pd.Series(np.array(tendances), index=['mean', 'mode', 'median', 'symetrie'])

In [5]:
def ecartType(df_column):
        return(np.sqrt((np.sum(np.power(df_column-mean(df_column),2)))/len(df_column)))

def variance(df_column):
        return(np.power(ecartType(df_column),2))

def getQuartiles(df_column):
        df_column = df_column.sort_values(ascending=True).reset_index(drop=True)
        return (df_column[len(df_column)//4], df_column[(len(df_column)//4)*3])
        
def ecartInterquartile(df_column):
        Q1, Q3 = getQuartiles(df_column)
        return Q3-Q1
        
def dispersion(df_column): #get outlier data and make it into a pandas series
        mesures = []
        standard_deviation = ecartType(df_column)
        variance_ = variance(df_column)
        inter_q = ecartInterquartile(df_column)
        quart = getQuartiles(df_column)
        mesures.append(standard_deviation)
        mesures.append(variance_)
        mesures.append(inter_q)
        mesures.append(min(df_column))
        mesures.append(quart)
        mesures.append(max(df_column))
        outliers = []
        for each in df_column:
                if each > quart[1]+1.5*inter_q or each < quart[0]-1.5*inter_q:
                        outliers.append(each)
        return (pd.Series(np.array(mesures), index=['ecart_type', 'variance', 'IQR', 'min','quartiles', 'max']), set(outliers))

In [6]:

def z_score(dataframe):
        dataframe = pd.DataFrame(dataframe)
        z_scores = []
        means = []
        std_devs = []
        for j in dataframe.columns:
                means.append(mean(dataframe[[j]]).values[0])
                std_devs.append(ecartType(dataframe[[j]]).values[0])
        for i in range(0, dataframe.shape[0]):
                row = []
                c = 0
                for j in dataframe.columns:
                        row.append(((dataframe[[j]].iloc[i]-means[c])/std_devs[c]).values[0])
                        c+=1
                z_scores.append(row)
        normalized = pd.DataFrame(z_score)
        return normalized
        

In [7]:
def correlation(df_column1, df_column2):
        try:
                N = np.int64(len(df_column1))
                xy_sum = np.int64(np.sum(df_column1*df_column2))
                top_sum = np.int64(N*xy_sum)
                sum_x = np.sum(df_column1)
                sum_y = np.sum(df_column2)
                bottom_x = len(df_column1)*np.sum(list(map(lambda x: x*x, df_column1)))
                bottom_y = len(df_column2)*np.sum(list(map(lambda x: x*x, df_column2)))
                return ((top_sum - (sum_x*sum_y))/np.sqrt((bottom_x - np.power(sum_x,2))*(bottom_y - np.power(sum_y,2))))
        except: print("impossible to calculate")

In [8]:
df = pd.read_csv("dataset1.csv")
df

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [9]:
data_num = df.select_dtypes(exclude=['object'])
data_obj = df.select_dtypes(exclude=['float64', 'int64'])
data_num.head()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1102,1,2,1,1,2.0,94,3,2,...,1,80,0,8,0,1,6,4,0,5
1,49,279,8,1,1,2,3.0,61,2,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1373,2,2,1,4,4.0,92,2,1,...,2,80,0,7,3,3,0,0,0,0
3,33,1392,3,4,1,5,4.0,56,3,1,...,3,80,0,8,3,3,8,7,3,0
4,27,591,2,1,1,7,1.0,40,3,1,...,4,80,1,6,3,3,2,2,2,2


In [10]:
data_obj.head()

,Attrition,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18,OverTime
0,Yes,Travel_Rarely,Sales,Life Sciences,Female,Sales Executive,Single,Y,Yes
1,No,Travel_Frequently,Research & Development,Life Sciences,Male,Research Scientist,Married,Y,No
2,Yes,Travel_Rarely,Research & Development,Other,Male,Laboratory Technician,Single,Y,Yes
3,No,Travel_Frequently,Research & Development,Life Sciences,Female,Research Scientist,Married,Y,Yes
4,No,Travel_Rarely,Research & Development,Medical,Male,Laboratory Technician,Married,Y,No


In [11]:
mode(data_obj.Attrition)

'No'

### A. Traitement des valeurs manquantes et aberrantes

In [12]:
def replace_missing(df_column, method = 'mode'):
        print(df_column.dtype)
        if df_column.dtype == 'object':
                if method != 'mode': print('this method cannot be used on non-numerical attributes')
                else: 
                        for row in df_column.items():
                                if pd.isnull(row[1]) : 
                                        if method == 'mode':
                                                df_column[row[0]] = mode(df_column)
        else:
                for row in df_column.items():
                        #print(df_column[0])
                        if pd.isnull(row[1]) : 
                                if method == 'mode':
                                        df_column[row[0]] = mode(df_column)
                                elif method == 'median':
                                        df_column[row[0]] = median(df_column)
                                elif method == 'mean':
                                        df_column[row[0]] = mean(df_column)
                                else: print("an error has occured")
        return df_column

def treat_outliers(dataframe, df_column, method = "delete"):
        outliers = dispersion(df_column)[1]
        df = dataframe.copy()
        #iterate through all rows
        for i in range(len(df_column)):
                if df_column.iloc[i] in outliers : 
                        if method == "delete":
                                df=df.drop([i])
                        elif method == "null":
                                df_column.iloc[i] = np.nan
                        elif method == "replace_mode":
                                df_column.iloc[i] = mode(df_column)
                        elif method == "replace_mean":
                                if df_column.dtype == "int64":
                                        df_column.iloc[i] = round(mean(df_column))
                                else: df_column.iloc[i] = mean(df_column)
                        elif method == "replace_median":
                                df_column.iloc[i] = median(df_column)
                                
                        else: print("la methode choisie est invalide")
        df[df_column.name] = df_column

        return df


In [13]:
#replace_missing(df.EnvironmentSatisfaction).isnull().sum()
treat_outliers(df, df.YearsInCurrentRole,method = "null").isnull().sum()
replace_missing(df.YearsInCurrentRole).isnull().sum()

float64


<ipython-input-5-9bec7b3bb58c>:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return (pd.Series(np.array(mesures), index=['ecart_type', 'variance', 'IQR', 'min','quartiles', 'max']), set(outliers))
C:\Users\Rania\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-12-cb93d16202b1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

0

### B. Réduction des données via la discretisation des données continues

In [14]:
# divide the dataset into n quantiles of the same amount of values/rows
# this means each group will have the same amount of values but different interval sizes
# we can reduce a group into a single row by either doing the mean, the median or the mode
def discretisation_effectifs(df_column, Q, method = "mean"):
        step = (len(df_column)//Q)+1
        column = df_column.sort_values(ascending=True).reset_index(drop=True)
        quantiles = []
        for i in range(0,len(df_column),step):
                quantiles.append(column.iloc[i:i+step])
        for i in range(len(quantiles)): 
                if method == "mean":
                        if df_column.dtype == "int64":
                                quantiles[i] = round(mean(quantiles[i]))
                        else:
                                quantiles[i] = mean(quantiles[i])
                elif method == "mode":
                        quantiles[i] = mode(quantiles[i])
                elif method == "median":
                        quantiles[i] = median(quantiles[i])
                else: print("la methode choisie est invalide")

        # we return the reduced column and then we just make another function to loop over the df?
        # but in this case we need to ask the chosen method for each column so idk about that lol 
        # they must also have the same number of quantiles too so
        return quantiles

In [15]:
#discretisation_effectifs(df.Age, 20, method = "mean")

In [16]:
# divide the dataset into n intervals of values
# this means each group will have the same interval sizes but different amount of values
# we can reduce a group into a single row by either doing the mean, the median or the mode
# if k value is not given, use the formula of huntsberger
import math
def discretisation_amplitude(df_column, K = 0, method = "mean"):
        if K==0:
                K = 1+3*math.log10(len(df_column))
        ## column = df_column.sort_values(ascending=True).reset_index(drop=True)
        interval_size = (max(df_column)- min(df_column))/K
        if df_column.dtype == "int64": interval_size = round(interval_size)
        intervals = []
        i = min(df_column)
        while i < max(df_column):
                intervals.append((i, i+interval_size))
                i+=interval_size
        intervals_values = {}
        for each in df_column:
                for interval in intervals:
                        if each > interval[0] and each < interval[1]:
                                if interval in intervals_values.keys():
                                        intervals_values[interval].append(each)
                                else: 
                                        intervals_values[interval] = [each]
        discretised = []
        for interval in intervals_values.keys():
                if method == "mean":
                        if df_column.dtype == "int64":
                                discretised.append(round(mean(intervals_values[interval])))
                        else: discretised.append(mean(intervals_values[interval]))
                elif method == "median":
                        discretised.append(median(intervals_values[interval]))
                elif method == "mode":
                        discretised.append(mode(intervals_values[interval]))
                else: print("methode choisie invalide")
        discretised.sort()
        return discretised

In [17]:
#discretisation_amplitude(df.Age)

### C. Réduction des données (redondances) horizontales et verticales

In [18]:
def detect_redundant(df):
        redundances = df.duplicated()
        redundant_columns = []
        for each in df.columns:
                if len(df[each].unique()) == 1 or len(df[each].unique()) == df.shape[0]:
                        redundant_columns.append(each)
        return df[redundances], redundant_columns


In [19]:
df.shape

(1470, 35)

In [20]:
detect_redundant(df)

(Empty DataFrame
 Columns: [Age, Attrition, BusinessTravel, DailyRate, Department, DistanceFromHome, Education, EducationField, EmployeeCount, EmployeeNumber, EnvironmentSatisfaction, Gender, HourlyRate, JobInvolvement, JobLevel, JobRole, JobSatisfaction, MaritalStatus, MonthlyIncome, MonthlyRate, NumCompaniesWorked, Over18, OverTime, PercentSalaryHike, PerformanceRating, RelationshipSatisfaction, StandardHours, StockOptionLevel, TotalWorkingYears, TrainingTimesLastYear, WorkLifeBalance, YearsAtCompany, YearsInCurrentRole, YearsSinceLastPromotion, YearsWithCurrManager]
 Index: []
 
 [0 rows x 35 columns],
 ['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'])

In [21]:
def del_row(df, index):
        return df.drop([index], axis = 0)

def del_column(df, index):
        if type(index) is str:
                if index in df.columns:
                        df = df.drop([index], axis = 1)
        elif type(int(index)) is int:
                df = df.drop(df.columns[index], axis = 1)
        return df


In [22]:
#del_column(df, "Age")

### D. Normalisation des données

In [32]:
# follow formula
def min_max_normalisation(df, min_, max_):
        if max_ < min_ : 
                print("min superior to max, please change values")
                return df
        else:
                normalized_df = []
                #df = df.select_dtypes(exclude=['object'])
                min_old = []
                max_old = []
                for column in df.columns:
                        min_old.append(min(df[column]))
                        max_old.append(max(df[column]))
                for i in range(df.shape[0]):
                        row = []
                        c = 0
                        for column in df.columns:
                                if df[column].dtype == 'object':
                                        row.append(df[column].iloc[i])
                                else:
                                        if max_old[c] != min_old[c]:
                                                row.append((((df[column].iloc[i]-min_old[c])/(max_old[c]- min_old[c]))*(max_- min_))+min_)
                                        else:
                                                row.append(min_)
                                c+=1
                        normalized_df.append(row)
                normalized = pd.DataFrame(normalized_df, columns = df.columns)
        return normalized
                                
                        

In [34]:
min_max_normalisation(df, 2,5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,3.642857,Yes,Travel_Rarely,4.147459,Sales,2.000000,2.75,Life Sciences,2,2.000000,...,2.0,2,2.0,2.600,2.0,2.0,2.450,2.857143,2.0,2.882353
1,4.214286,No,Travel_Frequently,2.380100,Research & Development,2.750000,2.00,Life Sciences,2,2.001451,...,5.0,2,3.0,2.750,3.5,4.0,2.750,3.500000,2.2,3.235294
2,3.357143,Yes,Travel_Rarely,4.729420,Research & Development,2.107143,2.75,Other,2,2.004354,...,3.0,2,2.0,2.525,3.5,4.0,2.000,2.000000,2.0,2.000000
3,3.071429,No,Travel_Frequently,4.770222,Research & Development,2.214286,4.25,Life Sciences,2,2.005806,...,4.0,2,2.0,2.600,3.5,4.0,2.600,3.500000,2.6,2.000000
4,2.642857,No,Travel_Rarely,3.050107,Research & Development,2.107143,2.00,Medical,2,2.008708,...,5.0,2,3.0,2.450,3.5,4.0,2.150,2.428571,2.4,2.352941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,3.285714,No,Travel_Frequently,3.679313,Research & Development,4.357143,2.75,Medical,2,4.989840,...,4.0,2,3.0,3.275,3.5,4.0,2.375,2.428571,2.0,2.529412
1466,3.500000,No,Travel_Rarely,3.097351,Research & Development,2.535714,2.00,Medical,2,4.991292,...,2.0,2,3.0,2.675,4.5,4.0,2.525,3.500000,2.2,3.235294
1467,2.642857,No,Travel_Rarely,2.113815,Research & Development,2.321429,3.50,Life Sciences,2,4.994194,...,3.0,2,3.0,2.450,2.0,4.0,2.450,2.428571,2.0,2.529412
1468,4.214286,No,Travel_Frequently,3.977810,Sales,2.107143,3.50,Medical,2,4.995646,...,5.0,2,2.0,3.275,3.5,3.0,2.675,3.285714,2.0,3.411765


In [42]:
#replace with the z score values, already done really but double check if it's the same formula
def z_score_normalisation(dataframe):
        dataframe = pd.DataFrame(dataframe)
        #dataframe = dataframe.select_dtypes(exclude=['object'])
        z_scores = []
        means = []
        std_devs = []
        for j in dataframe.columns:
                if dataframe[j].dtype !='object':
                        means.append(mean(dataframe[[j]]).values[0])
                        std_devs.append(ecartType(dataframe[[j]]).values[0])
        for i in range(0, dataframe.shape[0]):
                row = []
                c = 0
                for j in dataframe.columns:
                        if df[j].dtype == 'object':
                                row.append(df[j].iloc[i])
                        else:
                                row.append(((dataframe[[j]].iloc[i]-means[c])/std_devs[c]).values[0])
                                c+=1
                z_scores.append(row)
        normalized = pd.DataFrame(z_scores, columns = dataframe.columns)
        return normalized
        

In [43]:
z_score_normalisation(df)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.446350,Yes,Travel_Rarely,0.742527,Sales,-1.010909,-0.891688,Life Sciences,NaN,-1.701283,...,-1.584178,NaN,-0.932014,-0.421642,-2.171982,-2.493820,-0.164613,-0.008758,-0.679146,0.245834
1,1.322365,No,Travel_Frequently,-1.297775,Research & Development,-0.147150,-1.868426,Life Sciences,NaN,-1.699621,...,1.191438,NaN,0.241988,-0.164511,0.155707,0.338096,0.488508,0.889418,-0.368715,0.806541
2,0.008343,Yes,Travel_Rarely,1.414363,Research & Development,-0.887515,-0.891688,Other,NaN,-1.696298,...,-0.658973,NaN,-0.932014,-0.550208,0.155707,0.338096,-1.144294,-1.206326,-0.679146,-1.155935
3,-0.429664,No,Travel_Frequently,1.461466,Research & Development,-0.764121,1.061787,Life Sciences,NaN,-1.694636,...,0.266233,NaN,-0.932014,-0.421642,0.155707,0.338096,0.161947,0.889418,0.252146,-1.155935
4,-1.086676,No,Travel_Rarely,-0.524295,Research & Development,-0.887515,-1.868426,Medical,NaN,-1.691313,...,1.191438,NaN,0.241988,-0.678774,0.155707,0.338096,-0.817734,-0.607542,-0.058285,-0.595227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,-0.101159,No,Travel_Frequently,0.202082,Research & Development,1.703764,-0.891688,Medical,NaN,1.721670,...,0.266233,NaN,0.241988,0.735447,0.155707,0.338096,-0.327893,-0.607542,-0.679146,-0.314873
1466,0.227347,No,Travel_Rarely,-0.469754,Research & Development,-0.393938,-1.868426,Medical,NaN,1.723332,...,-1.584178,NaN,0.241988,-0.293077,1.707500,0.338096,-0.001333,0.889418,-0.368715,0.806541
1467,-1.086676,No,Travel_Rarely,-1.605183,Research & Development,-0.640727,0.085049,Life Sciences,NaN,1.726655,...,-0.658973,NaN,0.241988,-0.678774,-2.171982,0.338096,-0.164613,-0.607542,-0.679146,-0.314873
1468,1.322365,No,Travel_Frequently,0.546677,Sales,-0.887515,0.085049,Medical,NaN,1.728317,...,1.191438,NaN,-0.932014,0.735447,0.155707,-1.077862,0.325228,0.590026,-0.679146,1.086895


In [26]:
from scipy import stats
data = df.select_dtypes(exclude=['object'])
z_scores = pd.DataFrame(stats.zscore(data))
z_scores

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.446350,0.742527,-1.010909,-0.891688,NaN,-1.701283,NaN,1.383138,0.379672,-0.057788,...,-1.584178,NaN,-0.932014,-0.421642,-2.171982,-2.493820,-0.164613,-0.008758,-0.679146,0.245834
1,1.322365,-1.297775,-0.147150,-1.868426,NaN,-1.699621,NaN,-0.240677,-1.026167,-0.057788,...,1.191438,NaN,0.241988,-0.164511,0.155707,0.338096,0.488508,0.889418,-0.368715,0.806541
2,0.008343,1.414363,-0.887515,-0.891688,NaN,-1.696298,NaN,1.284725,-1.026167,-0.961486,...,-0.658973,NaN,-0.932014,-0.550208,0.155707,0.338096,-1.144294,-1.206326,-0.679146,-1.155935
3,-0.429664,1.461466,-0.764121,1.061787,NaN,-1.694636,NaN,-0.486709,0.379672,-0.961486,...,0.266233,NaN,-0.932014,-0.421642,0.155707,0.338096,0.161947,0.889418,0.252146,-1.155935
4,-1.086676,-0.524295,-0.887515,-1.868426,NaN,-1.691313,NaN,-1.274014,0.379672,-0.961486,...,1.191438,NaN,0.241988,-0.678774,0.155707,0.338096,-0.817734,-0.607542,-0.058285,-0.595227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,-0.101159,0.202082,1.703764,-0.891688,NaN,1.721670,NaN,-1.224807,1.785511,-0.057788,...,0.266233,NaN,0.241988,0.735447,0.155707,0.338096,-0.327893,-0.607542,-0.679146,-0.314873
1466,0.227347,-0.469754,-0.393938,-1.868426,NaN,1.723332,NaN,-1.175601,-1.026167,0.845911,...,-1.584178,NaN,0.241988,-0.293077,1.707500,0.338096,-0.001333,0.889418,-0.368715,0.806541
1467,-1.086676,-1.605183,-0.640727,0.085049,NaN,1.726655,NaN,1.038693,1.785511,-0.057788,...,-0.658973,NaN,0.241988,-0.678774,-2.171982,0.338096,-0.164613,-0.607542,-0.679146,-0.314873
1468,1.322365,0.546677,-0.887515,0.085049,NaN,1.728317,NaN,-0.142264,-1.026167,-0.057788,...,1.191438,NaN,-0.932014,0.735447,0.155707,-1.077862,0.325228,0.590026,-0.679146,1.086895


In [31]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler(feature_range=(2, 5))
names = data.columns
d = scaler.fit_transform(data)
scaled_df = pd.DataFrame(d, columns=names)
scaled_df.head()


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,3.642857,4.147459,2.000000,2.75,2.0,2.000000,3.0,4.742857,4.0,2.75,...,2.0,2.0,2.0,2.600,2.0,2.0,2.45,2.857143,2.0,2.882353
1,4.214286,2.380100,2.750000,2.00,2.0,2.001451,4.0,3.328571,3.0,2.75,...,5.0,2.0,3.0,2.750,3.5,4.0,2.75,3.500000,2.2,3.235294
2,3.357143,4.729420,2.107143,2.75,2.0,2.004354,5.0,4.657143,3.0,2.00,...,3.0,2.0,2.0,2.525,3.5,4.0,2.00,2.000000,2.0,2.000000
3,3.071429,4.770222,2.214286,4.25,2.0,2.005806,5.0,3.114286,4.0,2.00,...,4.0,2.0,2.0,2.600,3.5,4.0,2.60,3.500000,2.6,2.000000
4,2.642857,3.050107,2.107143,2.00,2.0,2.008708,2.0,2.428571,4.0,2.00,...,5.0,2.0,3.0,2.450,3.5,4.0,2.15,2.428571,2.4,2.352941
